In [3]:
import re
import pandas as pd
import time

### reviews_mecab 데이터 탐색

In [2]:
%%time
df_mecab = pd.read_csv('reviews_mecab_1.csv', encoding='utf-8', index_col=0)

CPU times: user 40.5 s, sys: 4.54 s, total: 45.1 s
Wall time: 1min 7s


In [ ]:
df_mecab = df_mecab.iloc[:50000]

In [3]:
df_mecab.head(2)

,reviewid,storeid,Link,Content,mecab
0,0,13992,https://blog.naver.com/sjswmfwow/221188193849,#광주떡갈비맛집 #광주송정역맛집 #빛고을떡갈비광주떡갈비 맛집빛고을떡갈비광주송정리 광...,"['ㅋㅋ 국물', 'ㅋㅋ 국물 으니', 'ㅋㅋ 육회', 'ㅋㅋ 육회 비빔밥', 'ㅎ..."
1,1,13992,https://blog.naver.com/white6346/221384202338,오리빠와 광주에서 부산 오는 날 먹고 온 광주 빛고을떡갈비급여행답게 검색 끝에 현지...,"['가능 시', '가능 시 겠', '각 인분', '각 인분 씩', '각 지역', '..."


In [4]:
# mecab 컬럼이 리스트로 있지 않은 행은 모두 삭제한다.
df_mecab = df_mecab[df_mecab.mecab != 'error']

In [5]:
# 필요한 컬럼만 남긴다.
df_mecab = df_mecab[['reviewid','storeid','mecab']]
df_mecab.shape

(199169, 2)

In [6]:
df_mecab.head(1)

,storeid,mecab
0,13992,"['ㅋㅋ 국물', 'ㅋㅋ 국물 으니', 'ㅋㅋ 육회', 'ㅋㅋ 육회 비빔밥', 'ㅎ..."


'mecab' 컬럼은 'Content' 컬럼을 (2,3)그램으로 형태소 분석한 결과를 리스트 형태로 저장한 컬럼이다.<br>
하지만 csv파일로 저장했다가 다시 불러왔으므로 현재 리스트가 스트링 타입으로 변환된 상태다.<br>
먼저 mecab 컬럼을 스트링으로 변환해준다.

In [7]:
type(df_mecab.mecab[0])

str

In [8]:
%%time
df_mecab['mecab'] = df_mecab['mecab'].apply(lambda x : eval(x))

CPU times: user 5min 7s, sys: 17.8 s, total: 5min 25s
Wall time: 5min 25s


In [9]:
# mecab 컬럼이 리스트 형태로 변환되었다.
type(df_mecab.mecab[0])

list

In [11]:
df_mecab.mecab[0][:10]

['ㅋㅋ 국물',
 'ㅋㅋ 국물 으니',
 'ㅋㅋ 육회',
 'ㅋㅋ 육회 비빔밥',
 'ㅎㅎ 군침',
 'ㅎㅎ 군침 돌',
 'ㅎㅎ 맨',
 'ㅎㅎ 맨 밥',
 'ㅎㅎ 밑반찬',
 'ㅎㅎ 밑반찬 넘']

### 가게/식당 감정 사전 데이터 탐색
- taste : good/bad/etc 맛에 대한 감정사전
- sanitation : good/bad/etc 가게 청결도에 대한 감정사전
- cs : good/bad/etc 서비스에 대한 감정사전
- atmosphere : good/bad/etc 가게 분위기/인테리어 대한 감정사전
- price : good/bad/etc 음식 가격에 대한 감정사전
- location : good/bad/etc 가게 위치에 대한 감정사전
- prefer : good/bad/etc 인기도에 대한 감정사전
- etc : good/bad/etc 그 외 기타 가게에 대한 감정사전
- restaurant_category_keywords : 가게 종류
- restaurant_category : 체인점 이름 

In [4]:
# 가게/식당 감정 사전 불러오기
df_dic = pd.read_csv("keywords_df.csv", encoding='utf-8', index_col=0)
df_dic.head(3)

,taste_good_keywords,taste_bad_keywords,taste_etc_keywords,sanitation_good_keywords,sanitation_bad_keywords,sanitation_etc_keywords,cs_good_keywords,cs_bad_keywords,cs_etc_keywords,atmosphere_good_keywords,...,location_bad_keywords,location_etc_keywords,etc_good_keywords,etc_bad_keywords,etc_etc_keywords,restaurant_category_keywords,restaurant_category,prefer_good_keywords,prefer_bad_keywords,prefer_etc_keywords
0,핵 맛집,딱딱,향 독특,잘 정돈,허름,정리 정돈,센스 굳,딱히,NaN,비주얼,...,헤맸,접근성,이쁜,개 빡,대기줄,뚝배기,맥도날드,종종 들릴,아쉽,NaN
1,진짜 맛있,나쁘,나쁘지 않,잘 진열,더럽,청결,가득 담겨,서비스 별로,NaN,인테리어 돋보였,...,힘들 게 찾,NaN,진짜 최고,감흥 없,누구 부담없이,피자,보노보노,줄 어마어마,이하 였,NaN
2,알싸,부족,달큰,상당히 깔끔,지저분,깨끗,인심 후하신,주차 길가,NaN,이쁜,...,힘들 게 찾아간,NaN,개웃,버리,놀이방 도 마련,평양냉면,쿠우 쿠우,자주 갈,아쉬운,NaN


감정 사전의 총 카테고리 별 단어 개수를 살펴본다.

In [16]:
total_tags = 0
for col in df_dic.columns:
    curr_col = df_dic[col]
    curr_col = curr_col[curr_col.notna()]
    curr_tags = ''
    try:
        curr_tags = ', '.join([curr_col[i] for i in range(5)])
    except:
        pass
    print(col, '키워드 개수:', curr_col.shape, curr_tags)
    total_tags += curr_col.shape[0]
    
print('\n전체 키워드 개수:',total_tags)
print('전체 카테고리 개수:', len(df_dic.columns))

taste_good_keywords 키워드 개수: (245,) 핵 맛집, 진짜 맛있, 알싸, 취향 저격, 입안 가득 육즙
taste_bad_keywords 키워드 개수: (70,) 딱딱, 나쁘, 부족, 비린, 눅눅
taste_etc_keywords 키워드 개수: (79,) 향 독특, 나쁘지 않, 달큰, 무난, 씹 식감
sanitation_good_keywords 키워드 개수: (8,) 잘 정돈, 잘 진열, 상당히 깔끔, 쾌적, 반짝반짝
sanitation_bad_keywords 키워드 개수: (4,) 
sanitation_etc_keywords 키워드 개수: (6,) 정리 정돈, 청결, 깨끗, 청소, 위생
cs_good_keywords 키워드 개수: (61,) 센스 굳, 가득 담겨, 인심 후하신, 이미 세팅, 서비스 덤
cs_bad_keywords 키워드 개수: (14,) 딱히, 서비스 별로, 주차 길가, 주차장 부족, 무시
cs_etc_keywords 키워드 개수: (0,) 
atmosphere_good_keywords 키워드 개수: (108,) 비주얼, 인테리어 돋보였, 이쁜, 정원, 쾌적
atmosphere_bad_keywords 키워드 개수: (15,) 별거 없, 협소, 정신 없는, 조명 너무 밝, 자리 비좁
atmosphere_etc_keywords 키워드 개수: (62,) 어린, 인증 샷, 인테리어 심플, 와이프, 분위기 도
price_good_keywords 키워드 개수: (48,) 가격 깔끔, 양 많, 가격대 싼, 쌈, 가성비 최고
price_bad_keywords 키워드 개수: (18,) 부담 스러운, 아쉬운 가격, 창 렬, 비쌌, 비쌈
price_etc_keywords 키워드 개수: (18,) 높 가격대, 놀라운 가격, 가격대, 가격 차이, 무한리필
location_good_keywords 키워드 개수: (8,) 가까운, 좋은 위치, 가까이, 명당자리, 위치 좋
location_bad_keywords 키워드 개수: (5,) 헤맸, 힘들 게 찾

In [2]:
(1239-9-145)/7

155.0

In [5]:
# 26개 컬럼을 리스트에 저장한다.
taste_good_count_list = df_dic.taste_good_keywords.dropna().to_list() 
taste_bad_count_list = df_dic.taste_bad_keywords.dropna().to_list()
taste_etc_count_list = df_dic.taste_etc_keywords.dropna().to_list()

sanitation_good_count_list = df_dic.sanitation_good_keywords.dropna().to_list()
sanitation_bad_count_list = df_dic.sanitation_bad_keywords.dropna().to_list()
sanitation_etc_count_list = df_dic.sanitation_etc_keywords.dropna().to_list()

cs_good_count_list = df_dic.cs_good_keywords.dropna().to_list()
cs_bad_count_list = df_dic.cs_bad_keywords.dropna().to_list()
cs_etc_count_list = df_dic.cs_etc_keywords.dropna().to_list()

atmosphere_good_count_list = df_dic.atmosphere_good_keywords.dropna().to_list()
atmosphere_bad_count_list = df_dic.atmosphere_bad_keywords.dropna().to_list()
atmosphere_etc_count_list = df_dic.atmosphere_etc_keywords.dropna().to_list()

price_good_count_list = df_dic.price_good_keywords.dropna().to_list()
price_bad_count_list = df_dic.price_bad_keywords.dropna().to_list()
price_etc_count_list = df_dic.price_etc_keywords.dropna().to_list()

location_good_count_list = df_dic.location_good_keywords.dropna().to_list()
location_bad_count_list = df_dic.location_bad_keywords.dropna().to_list()
location_etc_count_list = df_dic.location_etc_keywords.dropna().to_list()

etc_good_count_list = df_dic.etc_good_keywords.dropna().to_list()
etc_bad_count_list = df_dic.etc_bad_keywords.dropna().to_list()
etc_etc_count_list = df_dic.etc_etc_keywords.dropna().to_list()

restaurant_category_list = df_dic.restaurant_category_keywords.dropna().to_list()

prefer_good_count_list = df_dic.prefer_good_keywords.dropna().to_list()
prefer_bad_count_list = df_dic.prefer_bad_keywords.dropna().to_list()
prefer_etc_count_list = df_dic.prefer_etc_keywords.dropna().to_list()

total_category_list = [
        taste_good_count_list, taste_bad_count_list, taste_etc_count_list
       , sanitation_good_count_list, sanitation_bad_count_list, sanitation_etc_count_list
       , cs_good_count_list, cs_bad_count_list, cs_etc_count_list
       , atmosphere_good_count_list, atmosphere_bad_count_list, atmosphere_etc_count_list
       , price_good_count_list, price_bad_count_list, price_etc_count_list
       , location_good_count_list, location_bad_count_list, location_etc_count_list
       , etc_good_count_list, etc_bad_count_list, etc_etc_count_list
       , restaurant_category_list
       , prefer_good_count_list, prefer_bad_count_list, prefer_etc_count_list
                      ]

In [4]:
# df_dic의 26개 컬럼에서 restaurant_category만 제외하고 나머지를 total_category_list에 담는다.
len(total_category_list)

25

In [5]:
# NaN 값은 제외하고 잘 저장되었는지 개수를 다시 확인해본다. (restaurant_category를 제외하고 1,230개)
nums = 0
for li in total_category_list:
    nums += len(li)
nums

1230

### 라벨링

In [20]:
# 라벨링 해야 하는 형태소 단어가 몇 개인지 살펴본다.
total_mecabs = 0
for mecabs in df_mecab.mecab:
    total_mecabs += len(mecabs)
total_mecabs

196281705

현재 파일의 형태소 단어가 약 1억 9천개 이므로, 총 4개 파일을 다 더하면 대략 8억개다. <br>
이것을 리스트로 for문을 돌리면 시간이 너무 오래걸리므로 mecab 컬럼을 list 타입이 아닌 str 타입으로 바꿔준다. <br>
str 타입으로 바꾸면 list를 for문 돌릴 필요 없이 regular expression으로 한 번에 검색할 수 있기 때문에 빠르다.

In [23]:
%time
# mecab 리스트를 원소들 사이에 '&' 문자열을 추가하여 하나의 스트링으로 바꾼다.
df_mecab['mecab_join'] = df_mecab.mecab.apply(lambda x: '&'.join([word for word in x])) 

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.44 µs


In [25]:
# mecab_join의 맨 왼쪽/오른쪽에 '&' 문자를 추가한다.
df_mecab['mecab_join'] = df_mecab.mecab_join.apply(lambda x: '&' + x + '&') 

In [27]:
df_mecab.head(2)

,storeid,mecab,mecab_join
0,13992,"[ㅋㅋ 국물, ㅋㅋ 국물 으니, ㅋㅋ 육회, ㅋㅋ 육회 비빔밥, ㅎㅎ 군침, ㅎㅎ ...",&ㅋㅋ 국물&ㅋㅋ 국물 으니&ㅋㅋ 육회&ㅋㅋ 육회 비빔밥&ㅎㅎ 군침&ㅎㅎ 군침 돌&...
1,13992,"[가능 시, 가능 시 겠, 각 인분, 각 인분 씩, 각 지역, 각 지역 에, 각각 ...",&가능 시&가능 시 겠&각 인분&각 인분 씩&각 지역&각 지역 에&각각 매력&각각 ...


#### `col`에서 `keyword_list`의 `keyword`들을 포함하는 형태소 단어의 리스트를 반환하는 함수 `detect_mecab`을 정의한다.

In [6]:
def detect_mecab(col, keyword_list):
    matched_keyword_list = []
    
    for keyword in keyword_list:
        
        # mecab_join 컬럼은 '&형태소 단어&형태소 단어&형태소 단어&' 형태의 str이다.
        # keyword를 포함하는 형태소 단어 전체를 가져오는 regex를 compile 한 후,
        KEYWORD_RE = re.compile('[ ㄱ-ㅣ가-힣]*'+keyword+'[ ㄱ-ㅣ가-힣]*')
        
        # keyword를 발견하면 해당 keyword를 matched_keyword_list에 저장한다.
        if KEYWORD_RE.search(col) is not None:
            matched_keyword_list.append(keyword)
            
    return matched_keyword_list

In [57]:
# 25개 컬럼 중 한 컬럼을 먼저 돌려본다.
%timeit
loc_good_test = df_mecab.mecab_join.apply(detect_mecab, args=(location_good_count_list,))

# 함수로 한번에

In [9]:
def read_reviews_mecab(filepath):
    """
        filepath : reviews_mecab.csv
        return : 정제된 reviews_mecab (DataFrame)
    """
    
    # reviews_mecab.csv 파일을 읽고 필요 컬럼만 저장한다.
    df_mecab = pd.read_csv(filepath, encoding='utf-8', index_col=0)[['reviewid','storeid','mecab']]
    
    # mecab 컬럼이 리스트로 있지 않은 행은 모두 삭제한다.
    df_mecab = df_mecab[df_mecab.mecab != 'error']
    
    # mecab 컬럼의 타입을 str에서 list로 바꾼다.
    df_mecab['mecab'] = df_mecab['mecab'].apply(lambda x : eval(x))
    
    df_mecab['mecab'] = df_mecab.mecab.apply(lambda x: '&'.join([word for word in x])) 
    df_mecab['mecab'] = df_mecab.mecab.apply(lambda x: '&' + x + '&') 
    
    return df_mecab

In [ ]:
# 본격 라벨링 하는 코드!
%%time
df = read_reviews_mecab('./reviews_mecab_1.csv')

category_names = ['taste_good_count_list', 'taste_bad_count_list', 'taste_etc_count_list'
                 , 'sanitation_good_count_list', 'sanitation_bad_count_list', 'sanitation_etc_count_list'
                 , 'cs_good_count_list', 'cs_bad_count_list', 'cs_etc_count_list'
                 , 'atmosphere_good_count_list', 'atmosphere_bad_count_list', 'atmosphere_etc_count_list'
                 , 'price_good_count_list', 'price_bad_count_list', 'price_etc_count_list'
                 , 'location_good_count_list', 'location_bad_count_list', 'location_etc_count_list'
                 , 'etc_good_count_list', 'etc_bad_count_list', 'etc_etc_count_list'
                 , 'restaurant_category_list', 'prefer_good_count_list', 'prefer_bad_count_list', 'prefer_etc_count_list']

print('라벨링 시작:', time.asctime())
for i, category in zip(range(0,25),category_names):
    print(category, str(i+1)+'/25', time.asctime())
    df[category] = df.mecab.apply(detect_mecab, args=(total_category_list[i],))
print('라벨링 완료:', time.asctime())

df.drop('mecab', axis=1, inplace=True)
df.to_csv('./reviews_label_1.csv', encoding='utf-8')

taste_good_count_list 0//25


In [ ]:
%%time
df = read_reviews_mecab('./reviews_mecab_2.csv')

category_names = ['taste_good_count_list', 'taste_bad_count_list', 'taste_etc_count_list'
                 , 'sanitation_good_count_list', 'sanitation_bad_count_list', 'sanitation_etc_count_list'
                 , 'cs_good_count_list', 'cs_bad_count_list', 'cs_etc_count_list'
                 , 'atmosphere_good_count_list', 'atmosphere_bad_count_list', 'atmosphere_etc_count_list'
                 , 'price_good_count_list', 'price_bad_count_list', 'price_etc_count_list'
                 , 'location_good_count_list', 'location_bad_count_list', 'location_etc_count_list'
                 , 'etc_good_count_list', 'etc_bad_count_list', 'etc_etc_count_list'
                 , 'restaurant_category_list', 'prefer_good_count_list', 'prefer_bad_count_list', 'prefer_etc_count_list']

print('라벨링 시작:', time.asctime())
for i, category in zip(range(0,25),category_names):
    print(category, str(i+1)+'/25', time.asctime())
    df[category] = df.mecab.apply(detect_mecab, args=(total_category_list[i],))
print('라벨링 완료:', time.asctime())

df.drop('mecab', axis=1, inplace=True)
df.to_csv('./reviews_label_1.csv', encoding='utf-8')